In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm, load_target
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

In [3]:
URM_all = load_urm()

URM_train = sps.load_npz('Dataset/Split/URM_train.npz')
URM_test =  sps.load_npz('Dataset/Split/URM_test.npz')
URM_validation = sps.load_npz('Dataset/Split/URM_validation.npz')

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1481 ( 3.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 951 ( 2.3%) Users that have less than 1 test interactions


In [4]:
recommender_class = EASE_R_Recommender

In [5]:
import os

output_folder_path = "Experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 10
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"
cutoff_to_optimize = 10

In [6]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Categorical([None]),
    "normalize_matrix": Categorical([False]),
    "l2_norm": Real(low=200, high=300, prior='log-uniform'),
}

In [7]:
earlystopping_keywargs = {"validation_every_n": 15,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 5,
                          "validation_metric": metric_to_optimize,
                          }

In [8]:
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

#create a bayesian optimizer object, we pass the recommender and the evaluator
hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                           evaluator_validation=evaluator_validation)

In [9]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_all],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS ={}
)

In [10]:
hyperparameterSearch.search(recommender_input_args = recommender_input_args,
                            recommender_input_args_last_test = recommender_input_args_last_test,
                            hyperparameter_search_space = hyperparameters_range_dictionary,
                            n_cases = n_cases,
                            n_random_starts = n_random_starts,
                            save_model = "last",
                            output_folder_path = output_folder_path, # Where to save the results
                            output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                            metric_to_optimize = metric_to_optimize,
                            cutoff_to_optimize = cutoff_to_optimize,
                            )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': None, 'normalize_matrix': False, 'l2_norm': 225.7403645195658}
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 10.69 min
EvaluatorHoldout: Processed 40148 (100.0%) in 52.29 sec. Users per second: 768
SearchBayesianSkopt: New best config found. Config 0: {'topK': None, 'normalize_matrix': False, 'l2_norm': 225.7403645195658} - results: PRECISION: 0.0424479, PRECISION_RECALL_MIN_DEN: 0.0817520, RECALL: 0.0785336, MAP: 0.0193861, MAP_MIN_DEN: 0.0372616, MRR: 0.1416237, NDCG: 0.0732677, F1: 0.0551091, HIT_RATE: 0.3066404, ARHR_ALL_HITS: 0.1649593, NOVELTY: 0.0041089, AVERAGE_POPULARITY: 0.2452642, DIVERSITY_MEAN_INTER_LIST: 0.9088142, DIVERSITY_HERFINDAHL: 0.9908792, COVERAGE_ITEM: 0.1071939, COVERAGE_ITEM_HIT: 0.0437018, ITEMS_IN_GT: 0.9858408, COVERAGE_USER: 0.9644238, COVERAGE_USER_HIT: 0.2957313, USERS_IN_GT: 0.9644238, DIVERSITY_GINI: 0.0119725

In [ ]:
from Recommenders.DataIO import DataIO

#explore the results of the search
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

In [ ]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

In [ ]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

In [ ]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters